# Week 14: Poly Cube Mapping Attempt
A naive test on poly-cube mapping

## Table of Contents
- [0 - Packages and Resources](#0)
- [1 - Implementation](#1)
    - [*1.0 - The Example](#1-0)
    - [1.1 - Area Calculation](#1-1)
    - [1.2 - BFS](#1-2)
    - [1.3 - Fit a Plane](#1-3)
    - [1.4 - Intersection of Plane and Mesh](#1-4)
- [2 - Sample Outcome](#2)
    - [2.1 - One Round](#2-1)
    - [2.2 - Convergence](#2-2)
- [3 - Reference](#3)

<a name='0'></a>
## 0. Packages and Resources

In [25]:
# Packages
import numpy as np
import random
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Self-defined functions
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))
from util.util import distance_euclidean
from util.mesh.triangle.R3 import calculate_single_dihedral_angle, calculate_all_dihedral_angles, calculate_face_normals
from util.mesh.triangle.common import retrieve_all_edges
from util.triangle import calculate_area

# Visualization
import pyvista as pv
from pyvista import examples

<a name='1'></a>
## 1. Implementation

<a name='1-0'></a>
### *1.0 The Example

Here I use the sphere as an example.

In [131]:
faces.shape

(160, 3)

In [132]:
vertices.shape

(82, 3)

In [129]:
sphere = pv.Sphere(radius=5, theta_resolution=10, phi_resolution=10)

faces = sphere.faces.reshape((-1,4))[:, 1:4]
vertices = sphere.points
angles = calculate_all_dihedral_angles(faces, vertices)
edges = retrieve_all_edges(faces)

pl = pv.Plotter()
pl.add_mesh(sphere, show_edges=True, color="white")
pl.add_points(sphere.points, color="red", point_size=5)
# For monitor purpose
# pl.add_points(sphere.points[55], color="blue", point_size=20)
# pl.add_points(sphere.points[64], color="blue", point_size=20)
pl.show()

Widget(value="<iframe src='http://localhost:56023/index.html?ui=P_0x2d38462b1c0_15&reconnect=auto' style='widt…

In [123]:
class chen_2023:
    def __init__(self, vertices, faces, lambda1=1, lambda2=1, max_iter = 30):
        self.max_iter = max_iter
        self.vertex_num = vertices.shape[0]
        self.vertices = vertices
        self.faces = faces
        self.lambda1=lambda1
        self.lambda2=lambda2
        
        self.solution = self.vertices.copy()
        self.edges = retrieve_all_edges(faces)
    
    def activation(self, angle):
        if angle < (np.pi/2):
            return np.power(np.cos(angle), 2) / angle
        else:
            return np.power(np.cos(angle), 2)
        
    def loss_classification(self, x):
        X = x.reshape((self.vertex_num , 3))
        
        EB = 0
        for i, face in enumerate(self.faces):
            v1, v2, v3 = X[face[0]], X[face[1]], X[face[2]]
            a = v2 - v1
            b = v3 - v1
            cross_product = np.cross(a, b)
#             normal = cross_product / np.linalg.norm(cross_product)
            area = np.linalg.norm(cross_product) / 2
#             EB += area * (np.abs(normal) - 1)
            EB += np.sum(np.abs(cross_product)) - area
#         print(EB)
            
        EA = 0
        dihedral_angles = calculate_all_dihedral_angles(faces, X)
        for angle_value in dihedral_angles.values():
            EA += self.activation(angle_value)
            
        return self.lambda1*EB + self.lambda2*EA
    
    def optimize(self):
        x0 = np.ravel(self.solution)
        self.res = minimize(self.loss_classification, x0, options = {'maxiter': self.max_iter})
        self.solution = self.res.x.reshape((self.vertex_num, 3))
    
    def optimize_one_round(self):
        x0 = np.ravel(self.solution)
        self.res = minimize(self.loss_classification, x0, options = {'maxiter': 1})
        self.solution = self.res.x.reshape((self.vertex_num, 3))
    
    def optimize_test(self, num_iteration):
        for i in range(num_iteration):
            self.optimize_one_round()
            print(i)

In [73]:
test = chen_2023(vertices.copy(), faces.copy(), max_iter = 50)

In [74]:
test.optimize()

In [75]:
test.res

      fun: 427.93887090672087
 hess_inv: array([[ 1.02008689, -0.04347212, -0.10399234, ..., -0.02882962,
        -0.01807693,  0.16052585],
       [-0.04347212,  1.03181948,  0.09757949, ...,  0.03766918,
         0.02814298, -0.22320222],
       [-0.10399234,  0.09757949,  1.0394334 , ...,  0.08273077,
         0.06032672, -0.46547412],
       ...,
       [-0.02882962,  0.03766918,  0.08273077, ...,  1.00918357,
         0.01440198, -0.10218577],
       [-0.01807693,  0.02814298,  0.06032672, ...,  0.01440198,
         1.01085815, -0.09077972],
       [ 0.16052585, -0.22320222, -0.46547412, ..., -0.10218577,
        -0.09077972,  1.67492355]])
      jac: array([-2.93841934e+00, -1.20493698e+00,  2.79832840e+00, -3.50543976e-01,
       -9.09084320e-01,  8.98387909e-01,  2.54907990e+00,  3.26128006e+00,
       -6.83007050e+00, -4.35930634e+00,  1.36554718e-01, -1.90932465e+00,
        7.81411362e+00,  1.97052002e-01, -7.76561737e-01,  1.49336128e+01,
        1.67890549e+00, -3.67802811

In [76]:
mesh_update = pv.PolyData(test.solution, sphere.faces)

In [126]:
p = pv.Plotter()
p.add_mesh(mesh_update, show_edges=True, color="green", opacity=0.5)
p.add_mesh(sphere, show_edges=True, color="white", opacity=0.5)
p.add_points(mesh_update.points, color="red", point_size=5)
p.show()

Widget(value="<iframe src='http://localhost:56023/index.html?ui=P_0x2d38a4e9ee0_12&reconnect=auto' style='widt…

In [114]:
ship.is_all_triangles

True

In [108]:
ship = pv.read('input.ply')

In [115]:
faces = ship.faces.reshape((-1,4))[:, 1:4]
vertices = ship.points
angles = calculate_all_dihedral_angles(faces, vertices)
edges = retrieve_all_edges(faces)

In [111]:
vertices.shape

(256, 3)

In [112]:
faces.shape

(508, 3)

In [124]:
test_ship = chen_2023(vertices.copy(), faces.copy(), max_iter = 10)

In [125]:
test_ship.optimize_test(10)

KeyboardInterrupt: 

In [ ]:
air_plane_update = pv.PolyData(test_air_plane.solution, test_air_plane.faces)

In [128]:
p2 = pv.Plotter()
# p2.add_mesh(air_plane_update, show_edges=True, color="green", opacity=0.5)
p2.add_mesh(ship, show_edges=True, color="white", opacity=0.5)
p2.add_points(ship.points, color="red", point_size=5)
p2.show()

Widget(value="<iframe src='http://localhost:56023/index.html?ui=P_0x2d384626520_14&reconnect=auto' style='widt…